# Overview

This is a self-practice project for building a Hybrid Retrieval Augment Generative pipelie following Haystack and OpenAI tutorials.

## Prepare Environment

In [1]:
# install dependencies
!pip install haystack-ai chroma-haystack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pypr

## Download Dataset

In [2]:
# Download "History of the Standard Oil Company" from gutenberg porject
import urllib.request
urllib.request.urlretrieve("https://www.gutenberg.org/cache/epub/60692/pg60692.txt", "dataset.txt")

('dataset.txt', <http.client.HTTPMessage at 0x7a6c79a7ef50>)

## Setup OpenAI API Key

In [3]:
import os
os.environ["OPENAI_API_KEY"] = ""

## Build Document Indexing Pipeline

In [4]:
from haystack import Pipeline
from haystack_integrations.document_stores.chroma import ChromaDocumentStore
from haystack.components.converters import TextFileToDocument
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import OpenAIDocumentEmbedder
from haystack.components.writers import DocumentWriter

# create document store
document_store = ChromaDocumentStore()

# add pipeline componet
indexing_pipeline  = Pipeline()
indexing_pipeline.add_component("converter", TextFileToDocument())
indexing_pipeline.add_component("cleaner", DocumentCleaner())
indexing_pipeline.add_component("splitter", DocumentSplitter())
indexing_pipeline.add_component("embedder", OpenAIDocumentEmbedder())
indexing_pipeline.add_component("writer", DocumentWriter(document_store))

# connect pipeline
indexing_pipeline.connect("converter.documents", "cleaner.documents")
indexing_pipeline.connect("cleaner.documents", "splitter.documents")
indexing_pipeline.connect("splitter.documents", "embedder.documents")
indexing_pipeline.connect("embedder.documents", "writer.documents")

# preprocess dataset
indexing_pipeline.run(data={"sources": ["dataset.txt"]})

Calculating embeddings: 100%|██████████| 48/48 [00:20<00:00,  2.30it/s]


{'embedder': {'meta': {'model': 'text-embedding-ada-002',
   'usage': {'prompt_tokens': 392712, 'total_tokens': 392712}}},
 'writer': {'documents_written': 1512}}

## Build RAG Pipeline

In [5]:
# Build rag pipeline
from haystack.components.embedders import OpenAITextEmbedder
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator

# template for promt engineering
template = """Given these documents, answer the question.
              Documents:
              {% for doc in documents %}
                  {{ doc.content }}
              {% endfor %}
              Question: {{query}}
              Answer:"""

# add pipeline componet
rag_pipeline = Pipeline()
rag_pipeline.add_component("text_embedder", OpenAITextEmbedder())
rag_pipeline.add_component("retriever", ChromaEmbeddingRetriever(document_store))
rag_pipeline.add_component("prompt_builder", PromptBuilder(template=template))
rag_pipeline.add_component("llm", OpenAIGenerator())

# connect pipeline
rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever.documents", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")

🚅 Components
  - text_embedder: OpenAITextEmbedder
  - retriever: ChromaEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [6]:
def get_rag_reply(query):
    result = rag_pipeline.run(data={"prompt_builder": {"query":query}, "text_embedder": {"text": query}})
    return result["llm"]["replies"][0]

## Query And Get Reply

In [7]:
query = "Who is the founder of the standard oil company?"
reply = get_rag_reply(query)
print(reply)

John D. Rockefeller


In [8]:
query = "Why is standard oil company so successful?"
reply = get_rag_reply(query)
print(reply)

The Standard Oil Company is successful because it possesses qualities such as energy, intelligence, dauntlessness, ability, daring, and address. It has been strong in all great business qualities and has continuously adapted to new conditions as they arise. Additionally, it has secured special privileges and formed alliances with railroads to drive out rivals, control the output of oil, and regulate the price of oil. The company has also manipulated prices, stifled competition, and exercised power over prices with skill. It has a well-centralised authority, manages operations like partners in a business, collects valuable information, and ensures quick adaptability to new conditions. Furthermore, it has built a powerful trust that controls the oil industry almost absolutely and has expanded into various other interests such as railroads, shipping, and finance.
